<a href="https://colab.research.google.com/github/willismax/ML-in-Production-30-days-sharing/blob/main/notebook/13.Snorkel_%E8%A6%96%E8%A6%BA%E9%97%9C%E4%BF%82%E6%AA%A2%E6%B8%ACVRD%E7%AF%84%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Relationship Detection 視覺關係偵測





- 此範例源自 [snorkel-tutorials](https://github.com/snorkel-team/snorkel-tutorials/blob/master/visual_relation/visual_relation_tutorial.ipynb)，目的為對[視覺關係檢測 (VRD) 數據集](https://cs.stanford.edu/people/ranjaykrishna/vrd/)進行操作，專注於圖片內物件之間的關係分類任務。
- 通常圖片內容物都有物體之間的關聯性，定義描述為為`a subject <predictate> object`。
    - 例如，`person riding bicycle`，“person”和“bicycle”分別是主詞和受詞，“riding”是關係動詞。
- 以下圖示紅色框代表主題，而綠色框代表對象。該主詞（如踢）表示什麼關係連接主體和客體。

![](https://i.imgur.com/SMxT2C4.png)


## 設定專案環境

複製專案

In [ ]:
!git clone https://github.com/snorkel-team/snorkel-tutorials.git > clone_log.txt
!pip3 install snorkel

In [ ]:
%cd snorkel-tutorials/visual_relation

/content/snorkel-tutorials/visual_relation


*改寫專案檔案
- 因範例採用舊的pandas，已棄用的`df.as_matrix()`還在範例程式`snorkel-tutorials/visual_relation/model.py`中，為了符合現今環境，將改寫為`df.values` (model.py第135行附近)。
- 您可以直接執行以下`%%writefile`指令，或自行循路徑修正檔案。

In [ ]:
%%writefile /content/snorkel-tutorials/visual_relation/model.py

# fix _get_wordvec() df.as_matrix() to df.values (line:137)
import csv
from pathlib import Path
from typing import Tuple

import numpy as np
import pandas
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms

from snorkel.analysis import Scorer
from snorkel.classification import DictDataset, MultitaskClassifier, Operation, Task
from snorkel.classification.data import XDict, YDict


def union(bbox1, bbox2):
    """Create the union of the two bboxes.

    Parameters
    ----------
    bbox1
        Coordinates of first bounding box
    bbox2
        Coordinates of second bounding box

    Returns
    -------
    [y0, y1, x0, x1]
        Coordinates of union of input bounding boxes

    """
    y0 = min(bbox1[0], bbox2[0])
    y1 = max(bbox1[1], bbox2[1])
    x0 = min(bbox1[2], bbox2[2])
    x1 = max(bbox1[3], bbox2[3])
    return [y0, y1, x0, x1]


def crop_img_arr(img_arr, bbox):
    """Crop bounding box from image.

    Parameters
    ----------
    img_arr
        Image in array format
    bbox
        Coordinates of bounding box to crop

    Returns
    -------
    img_arr
        Cropped image

    """
    return img_arr[bbox[0] : bbox[1], bbox[2] : bbox[3], :]


class SceneGraphDataset(DictDataset):
    """Dataloader for Scene Graph Dataset."""

    def __init__(
        self,
        name: str,
        split: str,
        image_dir: str,
        df: pandas.DataFrame,
        image_size=224,
    ) -> None:
        self.image_dir = Path(image_dir)
        X_dict = {
            "img_fn": df["source_img"].tolist(),
            "obj_bbox": df["object_bbox"].tolist(),
            "sub_bbox": df["subject_bbox"].tolist(),
            "obj_category": df["object_category"].tolist(),
            "sub_category": df["subject_category"].tolist(),
        }
        Y_dict = {
            "visual_relation_task": torch.LongTensor(df["label"].to_numpy())
        }  # change to take in the rounded train labels
        super(SceneGraphDataset, self).__init__(name, split, X_dict, Y_dict)

        # define standard set of transformations to apply to each image
        self.transform = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def __getitem__(self, index: int) -> Tuple[XDict, YDict]:
        img_fn = self.X_dict["img_fn"][index]
        img_arr = np.array(Image.open(self.image_dir / img_fn))

        obj_bbox = self.X_dict["obj_bbox"][index]
        sub_bbox = self.X_dict["sub_bbox"][index]
        obj_category = self.X_dict["obj_category"][index]
        sub_category = self.X_dict["sub_category"][index]

        # compute crops
        obj_crop = crop_img_arr(img_arr, obj_bbox)
        sub_crop = crop_img_arr(img_arr, sub_bbox)
        union_crop = crop_img_arr(img_arr, union(obj_bbox, sub_bbox))

        # transform each crop
        x_dict = {
            "obj_crop": self.transform(Image.fromarray(obj_crop)),
            "sub_crop": self.transform(Image.fromarray(sub_crop)),
            "union_crop": self.transform(Image.fromarray(union_crop)),
            "obj_category": obj_category,
            "sub_category": sub_category,
        }

        y_dict = {name: label[index] for name, label in self.Y_dict.items()}
        return x_dict, y_dict

    def __len__(self):
        return len(self.X_dict["img_fn"])


class WordEmb(nn.Module):
    """Extract and concat word embeddings for obj and sub categories."""

    def __init__(self, glove_fn="data/glove/glove.6B.100d.txt"):
        super(WordEmb, self).__init__()

        self.word_embs = pandas.read_csv(
            glove_fn, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE
        )

    def _get_wordvec(self, word):
        return self.word_embs.loc[word].values

    def forward(self, obj_category, sub_category):
        obj_emb = self._get_wordvec(obj_category)
        sub_emb = self._get_wordvec(sub_category)
        embs = np.concatenate([obj_emb, sub_emb], axis=1)
        return torch.FloatTensor(embs)


# Classes and helper functions for defining classifier
def init_fc(fc):
    torch.nn.init.xavier_uniform_(fc.weight)
    fc.bias.data.fill_(0.01)


class FlatConcat(nn.Module):
    """Module that flattens and concatenates features"""

    def forward(self, *inputs):
        return torch.cat([input.view(input.size(0), -1) for input in inputs], dim=1)


# Helper functions to geenerate operations
def get_op_sequence():
    # define feature extractors for each of the (union, subject, and object) image crops
    union_feat_op = Operation(
        name="union_feat_op",
        module_name="feat_extractor",
        inputs=[("_input_", "union_crop")],
    )

    sub_feat_op = Operation(
        name="sub_feat_op",
        module_name="feat_extractor",
        inputs=[("_input_", "sub_crop")],
    )

    obj_feat_op = Operation(
        name="obj_feat_op",
        module_name="feat_extractor",
        inputs=[("_input_", "obj_crop")],
    )

    # define an operation to extract word embeddings for subject and object categories
    word_emb_op = Operation(
        name="word_emb_op",
        module_name="word_emb",
        inputs=[("_input_", "sub_category"), ("_input_", "obj_category")],
    )

    # define an operation to concatenate image features and word embeddings
    concat_op = Operation(
        name="concat_op",
        module_name="feat_concat",
        inputs=["obj_feat_op", "sub_feat_op", "union_feat_op", "word_emb_op"],
    )

    # define an operation to make a prediction over all concatenated features
    prediction_op = Operation(
        name="head_op", module_name="prediction_head", inputs=["concat_op"]
    )

    return [
        sub_feat_op,
        obj_feat_op,
        union_feat_op,
        word_emb_op,
        concat_op,
        prediction_op,
    ]


# Create model from pre loaded resnet cnn.
def create_model(resnet_cnn):
    # freeze the resnet weights
    for param in resnet_cnn.parameters():
        param.requires_grad = False

    # define input features
    in_features = resnet_cnn.fc.in_features
    feature_extractor = nn.Sequential(*list(resnet_cnn.children())[:-1])

    # initialize FC layer: maps 3 sets of image features to class logits
    WEMB_SIZE = 100
    fc = nn.Linear(in_features * 3 + 2 * WEMB_SIZE, 3)
    init_fc(fc)

    # define layers
    module_pool = nn.ModuleDict(
        {
            "feat_extractor": feature_extractor,
            "prediction_head": fc,
            "feat_concat": FlatConcat(),
            "word_emb": WordEmb(),
        }
    )

    # define task flow through modules
    op_sequence = get_op_sequence()
    pred_cls_task = Task(
        name="visual_relation_task",
        module_pool=module_pool,
        op_sequence=op_sequence,
        scorer=Scorer(metrics=["f1_micro"]),
    )
    return MultitaskClassifier([pred_cls_task])


Overwriting /content/snorkel-tutorials/visual_relation/model.py


## 1. 加載數據集



下載 [VRD 數據集]((https://cs.stanford.edu/people/ranjaykrishna/vrd/))並過濾包含至少一個動作謂詞的圖像，因為這些比幾何關係更難分類，如above或next to。
- 範例將訓練集、有效集和測試集加載為DataFrame：
    - `label`: 對象之間的關係。`0: RIDE, 1: CARRY, 2:OTHER動作謂詞`
    - `object_bbox`:  `[ymin, ymax, xmin, xmax]`
    - `object_category`
    - `source_img`
    - `subject_bbox`:  `[ymin, ymax, xmin, xmax]`
    - `subject_category`
- 數據集的採樣版本在訓練集、開發集和測試集上使用相同的 26 個數據。此設置旨在快速演示 Snorkel 如何處理此任務，而不是演示性能。

In [ ]:
import os
from utils import load_vrd_data

# setting sample=False will take ~3 hours to run (downloads full VRD dataset)
sample = True
is_test = os.environ.get("TRAVIS") == "true" or os.environ.get("IS_TEST") == "true"
df_train, df_valid, df_test = load_vrd_data(sample, is_test)

print("Train Relationships: ", len(df_train))
print("Dev Relationships: ", len(df_valid))
print("Test Relationships: ", len(df_test))

Train Relationships:  26
Dev Relationships:  26
Test Relationships:  26


In [ ]:
df_train.head()

,subject_category,object_category,subject_bbox,object_bbox,label,source_img
0,person,skateboard,"[46, 394, 59, 376]","[231, 397, 55, 191]",0,4788767927_236444fd03_b.jpg
1,person,ramp,"[111, 598, 354, 603]","[472, 1023, 1, 682]",2,4357208606_ce0f096a70_b.jpg
2,umbrella,person,"[188, 257, 383, 463]","[196, 367, 370, 444]",2,2820743071_dae064f6e7_o.jpg
3,umbrella,person,"[188, 257, 383, 463]","[196, 367, 370, 444]",2,2820743071_dae064f6e7_o.jpg
4,umbrella,person,"[196, 367, 370, 444]","[188, 257, 383, 463]",2,2820743071_dae064f6e7_o.jpg


請注意，訓練DataFrame將有一個全為 -1 的標籤字段。這表示該特定數據集缺少標籤。在本教程中，我們將通過在主體和客體的屬性上編寫標籤函數來為訓練集分配概率標籤！

## 2. 編寫Labeling Functions (LFs)

我們現在編寫標記函數來檢測邊界框對之間存在什麼關係。為此，我們可以將各種直覺編碼到標記函數中：

- 分類直覺：關於這些關係中通常涉及的主詞與受詞類別的知識（例如，person通常是動詞ride和的主詞carry）
- 空間直覺：關於主詞與動詞的相對位置的知識（例如，主詞通常高於動詞的受詞ride）

In [ ]:
RIDE = 0
CARRY = 1
OTHER = 2
ABSTAIN = -1

我們從編碼分類直覺的標記函數開始：我們使用關於共同的主題-客體類別對的知識RIDE，CARRY以及關於哪些主題或客體不太可能涉及這兩種關係的知識。

In [ ]:
from snorkel.labeling import labeling_function

# Category-based LFs
@labeling_function()
def lf_ride_object(x):
    if x.subject_category == "person":
        if x.object_category in [
            "bike",
            "snowboard",
            "motorcycle",
            "horse",
            "bus",
            "truck",
            "elephant",
        ]:
            return RIDE
    return ABSTAIN


@labeling_function()
def lf_carry_object(x):
    if x.subject_category == "person":
        if x.object_category in ["bag", "surfboard", "skis"]:
            return CARRY
    return ABSTAIN


@labeling_function()
def lf_carry_subject(x):
    if x.object_category == "person":
        if x.subject_category in ["chair", "bike", "snowboard", "motorcycle", "horse"]:
            return CARRY
    return ABSTAIN


@labeling_function()
def lf_not_person(x):
    if x.subject_category != "person":
        return OTHER
    return ABSTAIN

現在編碼空間直覺，其中包括測量邊界框之間的距離並比較它們的相對區域。

In [ ]:
YMIN = 0
YMAX = 1
XMIN = 2
XMAX = 3

In [ ]:
import numpy as np

# Distance-based LFs
@labeling_function()
def lf_ydist(x):
    if x.subject_bbox[XMAX] < x.object_bbox[XMAX]:
        return OTHER
    return ABSTAIN


@labeling_function()
def lf_dist(x):
    if np.linalg.norm(np.array(x.subject_bbox) - np.array(x.object_bbox)) <= 1000:
        return OTHER
    return ABSTAIN


def area(bbox):
    return (bbox[YMAX] - bbox[YMIN]) * (bbox[XMAX] - bbox[XMIN])


# Size-based LF
@labeling_function()
def lf_area(x):
    if area(x.subject_bbox) / area(x.object_bbox) <= 0.5:
        return OTHER
    return ABSTAIN

標記函數具有不同的經驗準確性和覆蓋範圍。由於我們選擇的關係中的類別不平衡，標記OTHER類的標記函數比RIDE或CARRY的標記函數具有更高的覆蓋率。這也反映了數據集中類的分佈。

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_ride_object,
    lf_carry_object,
    lf_carry_subject,
    lf_not_person,
    lf_ydist,
    lf_dist,
    lf_area,
]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_valid = applier.apply(df_valid)

100%|██████████| 26/26 [00:00<00:00, 5469.55it/s]


In [ ]:
from snorkel.labeling import LFAnalysis

Y_valid = df_valid.label.values
LFAnalysis(L_valid, lfs).lf_summary(Y_valid)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[-1  0  1  2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[-1  0  1  2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[-1  0  1  2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[-1  0  1  2] as keyword args. From version 1.0 (renaming of 0.25) passing these as posit

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_ride_object,0,[0],0.230769,0.230769,0.230769,5,1,0.833333
lf_carry_object,1,[1],0.076923,0.076923,0.076923,2,0,1.000000
lf_carry_subject,2,[1],0.038462,0.038462,0.038462,1,0,1.000000
lf_not_person,3,[2],0.307692,0.307692,0.038462,5,3,0.625000
lf_ydist,4,[2],0.576923,0.576923,0.307692,7,8,0.466667
lf_dist,5,[2],1.000000,0.846154,0.346154,13,13,0.500000
lf_area,6,[2],0.346154,0.346154,0.153846,5,4,0.555556


## 3. 訓練標籤模型

訓練`LabelModel`來為未標記的訓練集分配訓練標籤。

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=3, verbose=True)
label_model.fit(
    L_train, 
    seed=123, 
    lr=0.01, 
    log_freq=10, 
    n_epochs=100
    )

INFO:root:Computing O...
INFO:root:Estimating \mu...
  1%|          | 1/100 [00:00<00:10,  9.21epoch/s]INFO:root:[10 epochs]: TRAIN:[loss=0.474]
INFO:root:[20 epochs]: TRAIN:[loss=0.248]
INFO:root:[30 epochs]: TRAIN:[loss=0.092]
INFO:root:[40 epochs]: TRAIN:[loss=0.084]
INFO:root:[50 epochs]: TRAIN:[loss=0.062]
INFO:root:[60 epochs]: TRAIN:[loss=0.049]
INFO:root:[70 epochs]: TRAIN:[loss=0.042]
INFO:root:[80 epochs]: TRAIN:[loss=0.033]
INFO:root:[90 epochs]: TRAIN:[loss=0.026]
100%|██████████| 100/100 [00:00<00:00, 485.10epoch/s]
INFO:root:Finished Training


使用F1衡量模型
`F1 = 2 * (precision * recall) / (precision + recall)`


In [ ]:
label_model.score(L_valid, Y_valid, metrics=["f1_micro"])

{'f1_micro': 0.5769230769230769}

## 4. 訓練分類器

現在，您可以使用這些訓練標籤來訓練任何標準判別模型，例如現成的 [ResNet](https://github.com/KaimingHe/deep-residual-networks)，它應該學會在我們開發的 LF 之外進行泛化！

#### Create DataLoaders for Classifier

In [ ]:
from snorkel.classification import DictDataLoader
from model import SceneGraphDataset, create_model

df_train["labels"] = label_model.predict(L_train)

if sample:
    TRAIN_DIR = "data/VRD/sg_dataset/samples"
else:
    TRAIN_DIR = "data/VRD/sg_dataset/sg_train_images"

dl_train = DictDataLoader(
    SceneGraphDataset("train_dataset", "train", TRAIN_DIR, df_train),
    batch_size=16,
    shuffle=True,
)

dl_valid = DictDataLoader(
    SceneGraphDataset("valid_dataset", "valid", TRAIN_DIR, df_valid),
    batch_size=16,
    shuffle=False,
)

#### 定義模型架構

In [ ]:
import torchvision.models as models

# initialize pretrained feature extractor
cnn = models.resnet18(pretrained=True)
model = create_model(cnn)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:root:Created task: visual_relation_task
INFO:root:No cuda device available. Switch to cpu instead.
INFO:root:Created multi-task model MultitaskClassifier that contains task(s) {'visual_relation_task'} from 6 operations (0 shared) and 4 modules (0 shared).
INFO:root:No cuda device available. Switch to cpu instead.


### 訓練與評估模型

In [ ]:
from snorkel.classification import Trainer

trainer = Trainer(
    n_epochs=1,  # increase for improved performance
    lr=1e-3,
    checkpointing=True,
    checkpointer_config={"checkpoint_dir": "checkpoint"},
)
trainer.fit(model, [dl_train])

INFO:root:Save checkpoints at 'checkpoint' every 1.0 epochs.
INFO:root:Evaluating every 1.0 epochs.
INFO:root:Using optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)
INFO:root:Start training...
Epoch 0::  50%|█████     | 1/2 [00:04<00:04,  4.31s/it, model/all/train/loss=1.83, model/all/train/lr=0.001]INFO:root:checkpoint_runway condition has been met. Start checkpointing.
INFO:root:[MultitaskClassifier] Model saved in checkpoint/checkpoint_1.0.pth
INFO:root:Save checkpoint at 1.0 epochs at checkpoint/checkpoint_1.0.pth.
INFO:root:Save best model of metric model/all/train/loss at checkpoint/best_model_model_all_train_loss.pth
Epoch 0:: 100%|██████████| 2/2 [00:07<00:00,  3.55s/it, model/all/train/loss=1.72, model/all/train/lr=0.001]
INFO:root:Clear all checkpoints other than best so far.
INFO:root:Loading the best model from checkpoint/best_model_model_all_train_loss.pth.
INFO:root:[Mult

In [ ]:
model.score([dl_valid])

{'visual_relation_task/valid_dataset/valid/f1_micro': 0.34615384615384615}

- 我們已經成功訓練了一個視覺關係檢測模型！使用關於視覺關係中的對像如何相互作用的分類和空間直覺，我們能夠在多類分類設置中為 VRD 數據集中的對像對分配高質量的訓練標籤。

- 有關 Snorkel 如何用於視覺關係任務的更多信息，請參閱[ICCV 2019 論文](https://arxiv.org/abs/1904.11622)！